# Algorithme page rank en partant de valeur initiale

En théorie, en partant d'un graphe directionnel on peut construire la matrice adjacente à ce graphe et construire le page rank à partir d'une système à n équation(s) se resolvant assez simplement. Du moins, pour un graphe comportant un nombre limité de sommet / pages. Seulement, dans le cas de google, il y aurait une quantité de page à traiter qui serait bien trop grande, environ 130 000 milliards. Et un système d'équation à 130 000 milliards d'inconnues prendrait trop de temps et l'algorithme serait absolument inexploitable et ce même si l'algorithme est resolvable en temps polynomial O(E.n). Pour palier à cela on peut partir de valeur initiale et ainsi calculer le page rank de chacune des pages indépendament. 

In [1]:
def pageRank(pages, iterations=100):
    pageAmount = len(pages) 
    scores = {}
    
    for page in pages:
        scores[page] = 1 / pageAmount

    for i in range(iterations):
        newScores = {}
        
        for page in pages:
            newScores[page] = 0
            
            inPages = get_inbound_links(pages, page)

            sum_contributions = 0
            for link in inPages:
                nbOut = len(pages[link])
                if nbOut > 0: 
                    sum_contributions += scores[link] / nbOut
            
            pr = 0.15 / pageAmount + 0.85 * sum_contributions
            newScores[page] = pr
        
        scores = newScores

    return scores

def get_inbound_links(pages, page):
    inbound_links = []
    for src in pages:
        if page in pages[src]:
            inbound_links.append(src)

    return inbound_links


In [2]:

pages = {
    'A': ['B'],
    'B': ['C'],
    'C': ['A', 'B']
}

print(pageRank(pages, 20))


{'A': 0.2148106274731486, 'B': 0.3973996608253249, 'C': 0.38778971170152615}


## Explication du code

Dans la première partie du code, on applique donc la modification consistant à partir d'un page rank initial pour chaque page. Pour simplifier, le mieux est de partir de 1/n, n étant le nombres de pages.

``` python
 for page in pages:
    scores[page] = 1 / pageAmount
```

Ensuite on commence à boucler sur le nombre d'itérations voulues et crée les nouveaux rank de l'itération. A noter que plus le nombre d'itérations est grand, plus le ranking final convergera vers les valeurs pouvant être obtenue via la méthode du système d'équation. Une très légère marge "d'erreur" est permise au vu de l'efficacité de l'algorithme.


``` python
for i in range(iterations):
    newScores = {}
```

On boucle ensuite une deuxième fois, cette fois ci sur les pages qui est donc, en terme de structure de données, une liste de liste mais qui représente la matrice d'adjacence du graphe initial. Enfin, dans notre cas, on ne représente pas la matrice sous forme de de liens entrant et sortant mais juste sous la forme : "A possède un lien entrant vers B et vers C " -> A = [B, C]. On y recupère la liste des pages ayant des liens entrant vers la page sur laquelle on itère. Pour cela on crée une fonction ```get_inbound_links``` permettant de récupérer ces liens entrant en partant de la liste de page et de la page dont on veut récupérer les liens entrant.

```python
def get_inbound_links(pages, page):
    inbound_links = []
    for src in pages:
        if page in pages[src]:
            inbound_links.append(src)

    return inbound_links

#--------------------------------------
inPages = get_inbound_links(pages, page)

sum_contributions = 0
```

On effectue finalement une dernière boucle en itérant sur les pages ayant des liens entrants vers celle sur laquelle on itère. On y récupère le nombre de liens sortants. Dans le cas où les nombre de liens sortants excède 0, on actualise la "contribution" des pages ayant des liens entrant correspondant au score de cette page divisé par son nombre de liens sortants

```python
for link in inPages:
    nbOut = len(pages[link])
    if nbOut > 0: 
        sum_contributions += scores[link] / nbOut
```

Finalement on applique les constantes de notre équation :

```python
pr = 0.15 / pageAmount + 0.85 * sum_contributions
newScores[page] = pr
```